In [6]:
import json
import numpy as np
import pandas as pd
import os
import twitter

from twitter_auth import get_secret


In [7]:
f = "./woeids.json"

In [8]:
woeids = pd.read_json(f)

In [9]:
woeids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 7 columns):
country        467 non-null object
countryCode    466 non-null object
name           467 non-null object
parentid       467 non-null int64
placeType      467 non-null object
url            467 non-null object
woeid          467 non-null int64
dtypes: int64(2), object(5)
memory usage: 25.6+ KB


In [12]:
woeids.head()

,country,countryCode,name,parentid,placeType,url,woeid
0,,None,Worldwide,0,"{'code': 19, 'name': 'Supername'}",http://where.yahooapis.com/v1/place/1,1
1,Canada,CA,Winnipeg,23424775,"{'code': 7, 'name': 'Town'}",http://where.yahooapis.com/v1/place/2972,2972
2,Canada,CA,Ottawa,23424775,"{'code': 7, 'name': 'Town'}",http://where.yahooapis.com/v1/place/3369,3369
3,Canada,CA,Quebec,23424775,"{'code': 7, 'name': 'Town'}",http://where.yahooapis.com/v1/place/3444,3444
4,Canada,CA,Montreal,23424775,"{'code': 7, 'name': 'Town'}",http://where.yahooapis.com/v1/place/3534,3534


In [13]:
test = woeids.head()
print(test.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>country</th>
      <th>countryCode</th>
      <th>name</th>
      <th>parentid</th>
      <th>placeType</th>
      <th>url</th>
      <th>woeid</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td></td>
      <td>None</td>
      <td>Worldwide</td>
      <td>0</td>
      <td>{'code': 19, 'name': 'Supername'}</td>
      <td>http://where.yahooapis.com/v1/place/1</td>
      <td>1</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Canada</td>
      <td>CA</td>
      <td>Winnipeg</td>
      <td>23424775</td>
      <td>{'code': 7, 'name': 'Town'}</td>
      <td>http://where.yahooapis.com/v1/place/2972</td>
      <td>2972</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Canada</td>
      <td>CA</td>
      <td>Ottawa</td>
      <td>23424775</td>
      <td>{'code': 7, 'name': 'Town'}</td>
      <td>http://where.yahooapis.com/v1/place/3369</td>
      <td>3369</td>
    

In [14]:
woeids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 7 columns):
country        467 non-null object
countryCode    466 non-null object
name           467 non-null object
parentid       467 non-null int64
placeType      467 non-null object
url            467 non-null object
woeid          467 non-null int64
dtypes: int64(2), object(5)
memory usage: 25.6+ KB


In [15]:
woeids.loc[woeids['name'] == 'Charlotte']

,country,countryCode,name,parentid,placeType,url,woeid
342,United States,US,Charlotte,23424977,"{'code': 7, 'name': 'Town'}",http://where.yahooapis.com/v1/place/2378426,2378426


In [16]:
def lookup(name, country='United States'):
    name = name.title()
    country = country.title()
    try:
        return woeids.loc[(woeids['country'] == country) & (woeids['name'] == name), 'woeid'].iloc[0]
    except IndexError: print("Not a valid country/name combo.")

In [17]:
lookup('milwaukee')

2451822

In [18]:
twit_auth = get_secret()

In [19]:
consumer_key = twit_auth['CONSUMER_KEY']
consumer_secret = twit_auth['CONSUMER_SECRET']
access_token = twit_auth['ACCESS_TOKEN']
access_secret = twit_auth['ACCESS_SECRET']

In [20]:
api = twitter.Api(consumer_key, consumer_secret, access_token, access_secret)

In [47]:
def trending_by_geo(name, country="United States"):
    woeid = lookup(name, country)
    try:
        raw = [(x.name, x.volume) for x in api.GetTrendsWoeid(woeid)]
    except:
        print("Not a Twitter geographical category.")
    else:
        filtered = []
        for result in raw:
            if result[1] != None:
                filtered.append(result)
        results = pd.DataFrame(data=filtered, columns=["HashTag", "TweetVol"])
        results = results.sort_values("TweetVol", ascending=False)
        results.index = range(1, len(results)+1)
        results['TweetVol'] = results.TweetVol.apply(lambda x : "{:,}".format(x))
#         print("Top Trending HashTags in {}".format(name.title()))
        return results

In [48]:
result = trending_by_geo("Seattle")

In [49]:
print(result.to_json(orient='index'))

{"1":{"HashTag":"X \u00c6 A-12","TweetVol":"1,115,238"},"2":{"HashTag":"Adele","TweetVol":"880,506"},"3":{"HashTag":"Grimes","TweetVol":"370,999"},"4":{"HashTag":"#2YearsWithSingularity","TweetVol":"202,386"},"5":{"HashTag":"Cinco de Mayo","TweetVol":"181,102"},"6":{"HashTag":"Popular","TweetVol":"159,593"},"7":{"HashTag":"#Chromatica","TweetVol":"139,257"},"8":{"HashTag":"#ExposeCBS","TweetVol":"120,296"},"9":{"HashTag":"#NationalNursesDay","TweetVol":"103,112"},"10":{"HashTag":"Kraftwerk","TweetVol":"70,181"},"11":{"HashTag":"Florian Schneider","TweetVol":"57,300"},"12":{"HashTag":"Milano","TweetVol":"56,328"},"13":{"HashTag":"#JusticeForAhmaud","TweetVol":"47,901"},"14":{"HashTag":"115 YEARS","TweetVol":"31,806"},"15":{"HashTag":"Grandma Killer","TweetVol":"24,144"},"16":{"HashTag":"Ashley Benson","TweetVol":"20,280"},"17":{"HashTag":"New Blacks","TweetVol":"19,583"},"18":{"HashTag":"#Banksy","TweetVol":"17,492"},"19":{"HashTag":"#NS10vs10","TweetVol":"17,219"},"20":{"HashTag":"#Clo